<a href="https://colab.research.google.com/github/Tejas325-creator/Chemathon-2025/blob/main/ML_MODEL_FOR_CHLOR_A_AND_PLASTIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [4]:
df  = pd.read_csv('/content/chla_plastic_full_dataset.csv')

In [5]:
print(df.shape)
df.head()

(520, 19)


,spectrum_id,deploy_id,datetime_utc,gps_lat,gps_lon,boat_speed_mps,sensor_model,wavelengths_nm,I_raw,white_ref,dark_ref,reflectance,chl_a_mg_m3_lab,turbidity_NTU,image_id,class_label,polymer_type,size_class,confidence
0,S_0001,DEP2025_006,2025-09-30T08:00:00,12.941483,77.440159,2.26,Cubert Firefly,400;410;420;430;440;450;460;470;480;490;500;51...,0.8197;0.1306;0.4183;0.6222;0.1087;0.6200;0.59...,0.9242;0.9744;0.9476;0.9493;0.9984;0.9783;0.92...,0.0068;0.0382;0.0039;0.0084;0.0419;0.0240;0.04...,0.8861;0.0987;0.4391;0.6523;0.0698;0.6245;0.62...,46.69,47.09,IMG_0001,foam,NaN,NaN,0.72
1,S_0002,DEP2025_019,2025-09-30T08:00:15,12.940438,77.575387,2.14,Specim IQ,400;410;420;430;440;450;460;470;480;490;500;51...,0.2533;0.7435;0.8929;0.8307;0.5115;0.1410;0.49...,0.9762;0.9268;0.9933;0.9226;0.9838;0.9751;0.99...,0.0245;0.0129;0.0150;0.0129;0.0208;0.0151;0.00...,0.2404;0.7995;0.8973;0.8990;0.5095;0.1311;0.49...,10.48,43.13,IMG_0002,plastic,PE,meso,1.00
2,S_0003,DEP2025_006,2025-09-30T08:00:30,12.984427,77.514587,1.52,Specim IQ,400;410;420;430;440;450;460;470;480;490;500;51...,0.5400;0.6505;0.3167;0.4362;0.7624;0.2756;0.14...,0.9868;0.9442;0.9751;0.9891;0.9946;0.9326;0.99...,0.0093;0.0414;0.0396;0.0369;0.0230;0.0317;0.03...,0.5429;0.6747;0.2962;0.4194;0.7611;0.2707;0.12...,3.67,44.67,IMG_0003,foam,NaN,NaN,0.67
3,S_0004,DEP2025_004,2025-09-30T08:00:45,12.880220,77.597647,2.31,Cubert Firefly,400;410;420;430;440;450;460;470;480;490;500;51...,0.3840;0.2232;0.8251;0.5443;0.4957;0.8821;0.38...,0.9892;0.9457;0.9004;0.9639;0.9287;0.9482;0.99...,0.0186;0.0422;0.0037;0.0373;0.0439;0.0404;0.02...,0.3764;0.2004;0.9160;0.5471;0.5106;0.9271;0.37...,33.81,13.41,IMG_0004,foam,NaN,NaN,0.70
4,S_0005,DEP2025_008,2025-09-30T08:01:00,12.938174,77.567171,0.57,Cubert Firefly,400;410;420;430;440;450;460;470;480;490;500;51...,0.5301;0.2858;0.6530;0.4162;0.2078;0.1090;0.12...,0.9111;0.9128;0.9125;0.9121;0.9227;0.9448;0.94...,0.0350;0.0261;0.0421;0.0460;0.0174;0.0338;0.02...,0.5652;0.2928;0.7019;0.4275;0.2102;0.0826;0.10...,22.46,44.78,IMG_0005,other,NaN,NaN,0.95


In [6]:
# Parse wavelength array (same across rows)
wavelengths = np.array(list(map(int, df['wavelengths_nm'][0].split(';'))))

# Convert reflectance string to array
def parse_reflectance(row):
    return np.array(list(map(float, row.split(';'))))

df['reflectance_array'] = df['reflectance'].apply(parse_reflectance)

# Helper: get reflectance at nearest wavelength
def get_R(ref_array, target_wavelength):
    idx = (np.abs(wavelengths - target_wavelength)).argmin()
    return ref_array[idx]

# Compute NIR/Red ratio
def compute_NIR_Red(ref_array):
    R_NIR = get_R(ref_array, 708)   # NIR band ~708 nm
    R_Red = get_R(ref_array, 665)   # Red band ~665 nm
    ratio = R_NIR / (R_Red + 1e-8)  # avoid division by zero
    return ratio

df['NIR_Red_ratio'] = df['reflectance_array'].apply(compute_NIR_Red)

# Save enriched dataset
df.to_csv("chla_plastic_with_NIR_Red.csv", index=False)

print("✅ NIR/Red ratio column added and dataset saved as chla_plastic_with_NIR_Red.csv")


✅ NIR/Red ratio column added and dataset saved as chla_plastic_with_NIR_Red.csv


In [7]:
df.head()

,spectrum_id,deploy_id,datetime_utc,gps_lat,gps_lon,boat_speed_mps,sensor_model,wavelengths_nm,I_raw,white_ref,...,reflectance,chl_a_mg_m3_lab,turbidity_NTU,image_id,class_label,polymer_type,size_class,confidence,reflectance_array,NIR_Red_ratio
0,S_0001,DEP2025_006,2025-09-30T08:00:00,12.941483,77.440159,2.26,Cubert Firefly,400;410;420;430;440;450;460;470;480;490;500;51...,0.8197;0.1306;0.4183;0.6222;0.1087;0.6200;0.59...,0.9242;0.9744;0.9476;0.9493;0.9984;0.9783;0.92...,...,0.8861;0.0987;0.4391;0.6523;0.0698;0.6245;0.62...,46.69,47.09,IMG_0001,foam,NaN,NaN,0.72,"[0.8861, 0.0987, 0.4391, 0.6523, 0.0698, 0.624...",0.171053
1,S_0002,DEP2025_019,2025-09-30T08:00:15,12.940438,77.575387,2.14,Specim IQ,400;410;420;430;440;450;460;470;480;490;500;51...,0.2533;0.7435;0.8929;0.8307;0.5115;0.1410;0.49...,0.9762;0.9268;0.9933;0.9226;0.9838;0.9751;0.99...,...,0.2404;0.7995;0.8973;0.8990;0.5095;0.1311;0.49...,10.48,43.13,IMG_0002,plastic,PE,meso,1.00,"[0.2404, 0.7995, 0.8973, 0.899, 0.5095, 0.1311...",0.827795
2,S_0003,DEP2025_006,2025-09-30T08:00:30,12.984427,77.514587,1.52,Specim IQ,400;410;420;430;440;450;460;470;480;490;500;51...,0.5400;0.6505;0.3167;0.4362;0.7624;0.2756;0.14...,0.9868;0.9442;0.9751;0.9891;0.9946;0.9326;0.99...,...,0.5429;0.6747;0.2962;0.4194;0.7611;0.2707;0.12...,3.67,44.67,IMG_0003,foam,NaN,NaN,0.67,"[0.5429, 0.6747, 0.2962, 0.4194, 0.7611, 0.270...",1.309105
3,S_0004,DEP2025_004,2025-09-30T08:00:45,12.880220,77.597647,2.31,Cubert Firefly,400;410;420;430;440;450;460;470;480;490;500;51...,0.3840;0.2232;0.8251;0.5443;0.4957;0.8821;0.38...,0.9892;0.9457;0.9004;0.9639;0.9287;0.9482;0.99...,...,0.3764;0.2004;0.9160;0.5471;0.5106;0.9271;0.37...,33.81,13.41,IMG_0004,foam,NaN,NaN,0.70,"[0.3764, 0.2004, 0.916, 0.5471, 0.5106, 0.9271...",4.211449
4,S_0005,DEP2025_008,2025-09-30T08:01:00,12.938174,77.567171,0.57,Cubert Firefly,400;410;420;430;440;450;460;470;480;490;500;51...,0.5301;0.2858;0.6530;0.4162;0.2078;0.1090;0.12...,0.9111;0.9128;0.9125;0.9121;0.9227;0.9448;0.94...,...,0.5652;0.2928;0.7019;0.4275;0.2102;0.0826;0.10...,22.46,44.78,IMG_0005,other,NaN,NaN,0.95,"[0.5652, 0.2928, 0.7019, 0.4275, 0.2102, 0.082...",1.384665


In [11]:
# Step 3: Define Features (X) and Target (y)
X = df[['NIR_Red_ratio']].values   # Feature for Chl-a detection
y = df['chl_a_mg_m3_lab'].values   # Ground-truth label (mg/m³)

print("Feature shape:", X.shape)
print("Label shape:", y.shape)

Feature shape: (520, 1)
Label shape: (520,)


In [12]:
# First split into train+val and test
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42
)

# Then split train+val into train and val
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.176, random_state=42
)
# 0.176 ≈ 15% of total, so final: 70/15/15

print("Train size:", X_train.shape[0])
print("Validation size:", X_val.shape[0])
print("Test size:", X_test.shape[0])


Train size: 364
Validation size: 78
Test size: 78
